# Tensor 编程练习

请尝试使用学到的tensor知识，完成下述问题。

In [1]:
import torch
import cv2
img = cv2.imread('./template.jpg')

### 1. tensor的创建、属性修改、索引、拼接、维度调整等

（1）使用opencv读取的图像默认是BGR三通道的ndarrays格式，数据类型是uint8。基于pytorch实现的神经网络往往要求输入数据使用float32类型，请创建一个和img具有相同内容的tensor，并将其数据类型调整为float32。

In [2]:
img_t = torch.tensor(img, dtype=torch.float32, device='cpu')

（2）请取出该tensor表示的图像的上半部分，即对于尺寸为（48，48，3）的图像取出前24行得到尺寸为（24，48，3）的tensor，命名为img_top。类似地，请再取出图像的下半部分，得到另一个尺寸为（24，48，3）的tensor，命名为img_down。

In [3]:
img_top = img_t[:24]
img_down = img_t[24:]

（3）请将img_down置于img_top上方对二者进行拼接，得到尺寸为（48，48，3）的tensor，命名为img_cat。


In [4]:
img_cat = torch.cat([img_down, img_top], dim=0)

（4）在pytorch实现的神经网络中，往往要求输入图像具有(batch_size, channel, height, width)的维度顺序。请将拼接后的img_cat从(height, width, channel)的维度顺序调整为(batch_size, channel, height, width)的顺序，命名为img_pytorch, 其中batch_size应为1。

In [5]:
img_pytorch = img_cat.permute(2, 0, 1).unsqueeze(0)

### 2. tensor的数学运算

（1）提供的模板图像是一幅黑白图像，可以很容易地根据三通道值之和是否为0来区分背景和前景。请据此得到img_pytorch的前景mask，命名为img_mask，img_mask应具有(batch_size, height, width)的维度，并且数据类型应为float32。

In [6]:
mask = torch.zeros(48, 48, dtype=torch.float32, device='cpu')
for i in range(48):
    for j in range(48):
        if (img_pytorch[0][0][i][j] + img_pytorch[0][1][i][j] + img_pytorch[0][2][i][j]):
            mask[i][j] = 255
img_mask = mask.unsqueeze(0)

（2）请对img_pytorch的前景部分添加在[-50, 0]之间均匀分布的随机噪声，结果命名为img_noise，可以使用前面获得的前景mask。

In [7]:
noise = mask - torch.rand(48, 48) * 50
for i in range(48):
    for j in range(48):
        if noise[i][j] < 0:
            noise[i][j] = 0
img_noise = img_pytorch - noise